In [2]:
'''
Requests the AWS Lambda function URL
IMPORTANT: Set the invoke mode of the Lambda function to RESPONSE_STREAM
'''

import json
import requests
from botocore.awsrequest import AWSRequest
from botocore.auth import SigV4Auth
from botocore.credentials import Credentials
import boto3

# Initialize a session using Boto3
session = boto3.Session(profile_name='renalworks-bedrock') # Replace the profile name (can be found for Windows users in /Users/{YOUR_USERNAME}/.aws/config) accordingly 
credentials = session.get_credentials().get_frozen_credentials()
region = session.region_name

# AWS Lambda function URL
lambda_function_url = "https://o5a4hq7eohvppotol5dmcdjrzi0vslwy.lambda-url.us-east-1.on.aws/"

# Request payload
payload = {
  "responseData": [
    "id,note,type,details,fhir_id,id_patient,created_at,modified_at,created_by,modified_by,id_tenant,id_branch,id_dialysis,manually_created,hd_reading_id,lab_order_id,,,,,,,,,,",
    "11989,'pt stable, complete 4 hour hd, no complaint',HD,Post-dialysis,694417,1951,54:09.4,,233,,100,10000,9042,FALSE,,,,,,,,,,,,",
    "11976,'pt stable, no sign of infection, no sign of covid',HD,Pre-dialysis,694027,1951,02:56.5,,233,,100,10000,9042,FALSE,,,,,,,,,,,,",
    "11859,'pt stable and afebrile\nno sign and symptom of infection seen\nno sign and symptom of edema seen\npt on profiling linear, start at 1.15L/hr',HD,Pre-dialysis,690742,1951,19:14.8,,172,,100,10000,8961,FALSE,,,,,,,,,,,,"
  ],
  "category": "clinical-note",
  "userInput": "How is the patient doing?",
  "previousSummary": ""
}

# Create an AWSRequest object
request = AWSRequest(
    method="POST",
    url=lambda_function_url,
    data=json.dumps(payload),
    headers={
        "Content-Type": "application/json"
    }
)

# Sign the request using SigV4
SigV4Auth(credentials, "lambda", region).add_auth(request)

# Send the signed request using requests
with requests.post(lambda_function_url, headers=dict(request.headers.items()), data=request.body, stream=True) as response:
    print(response.headers)
    if response.status_code == 200:
        for chunk in response.iter_content(decode_unicode=True):
             print(chunk, end='', flush=True)

        '''
        FOR USE ONLY IF JSON IS SENT AS BINARY DATA
        # Iterate over the response content     
        buffer = b''
        for chunk in response.iter_content(decode_unicode=True):
            buffer += chunk

            try:
                # Once the whole json chunk comes, we can decode it and get the body
                parsed_json = json.loads(buffer.decode('utf-8'))
                print(parsed_json.get("body"), end='')
                buffer = b''
            except:
                 continue
        '''
    else: 
            print(f"Failed to connect, status code: {response.status_code}")

{'Date': 'Fri, 23 Aug 2024 06:32:38 GMT', 'Content-Type': 'text/plain; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-amzn-RequestId': '9ba9c873-1728-4740-87eb-e9feaaad17d5', 'x-amzn-Remapped-server': 'uvicorn', 'X-Amzn-Trace-Id': 'root=1-66c82d02-0622543316989e7a4b7bfda2;parent=05176db4b287a2de;sampled=0;lineage=44f17ad1:0', 'x-amzn-Remapped-date': 'Fri, 23 Aug 2024 06:32:38 GMT'}
Based on the clinical notes provided, I can summarize the patient's condition as follows:
1. <b>Overall Status</b>:     <ul>         <li>The patient is generally stable across multiple dialysis sessions</li>         <li>No major complaints or complications reported</li>     </ul>
2. <b>Dialysis Treatment</b>:     <ul>         <li>Patient completed a 4-hour hemodialysis session successfully</li>         <li>Patient is on profiling linear dialysis, starting at 1.15L/hr</li>     </ul>
3. <b>Infection Status</b>:     <ul>         <li>No signs of infection observed</li>         <li

In [68]:
'''
Requests the local Flask backend
'''

import requests

def stream_response(url):
    with requests.get(url, stream=True) as response:
        if response.status_code == 200:
            # Iterate over the response content as it arrives
            for chunk in response.iter_content(decode_unicode=True):
                if chunk:
                    print(chunk, end='', flush=True)
        else: 
            print(f"Failed to connect, status code: {response.status_code}")

stream_response('http://localhost:5000')

{"statusCode": 200, "headers": {"Content-Type": "application/json", "Access-Control-Allow-Headers": "*", "Access-Control-Allow-Origin": "*", "Access-Control-Allow-Methods": "*"}}
Based on the clinical notes provided, I can summarize the patient's condition as follows:
1. <b>Overall Status</b>:     <ul>         <li>The patient is generally stable across multiple dialysis sessions</li>         <li>No major complaints or complications reported</li>     </ul>
2. <b>Dialysis Sessions</b>:     <ul>         <li>Patient completed a 4-hour hemodialysis session without issues</li>         <li>Pre-dialysis notes indicate patient was stable before treatments</li>         <li>One session used profiling linear, starting at 1.15L/hr</li>     </ul>
3. <b>Infection Status</b>:     <ul>         <li>No signs of infection observed</li>         <li>Specifically, no signs of COVID-19 noted</li>         <li>Patient reported as afebrile (no fever)</li>     </ul>
4. <b>Other Observations</b>:     <ul>         

In [13]:
'''
Requests the APIGateway integrated with the response-streaming Lambda
- This does NOT use authorization, so both the APIGateway's resource and Lambda's fURL must not require IAM authorization
- Does NOT stream for REST and HTTP APIs (the Transfer-Encoding:chunked header is removed by APIGateway)
'''

import requests

apigateway_url = "https://4spxcpvgei.execute-api.us-east-1.amazonaws.com/dev/text-summarization"
# apigateway_url = "https://f3dmrc6b5j.execute-api.us-east-1.amazonaws.com/dev/text-summarization"

payload = {
  "responseData": [
    "id,note,type,details,fhir_id,id_patient,created_at,modified_at,created_by,modified_by,id_tenant,id_branch,id_dialysis,manually_created,hd_reading_id,lab_order_id,,,,,,,,,,",
    "11989,'pt stable, complete 4 hour hd, no complaint',HD,Post-dialysis,694417,1951,54:09.4,,233,,100,10000,9042,FALSE,,,,,,,,,,,,",
    "11976,'pt stable, no sign of infection, no sign of covid',HD,Pre-dialysis,694027,1951,02:56.5,,233,,100,10000,9042,FALSE,,,,,,,,,,,,",
    "11859,'pt stable and afebrile\nno sign and symptom of infection seen\nno sign and symptom of edema seen\npt on profiling linear, start at 1.15L/hr',HD,Pre-dialysis,690742,1951,19:14.8,,172,,100,10000,8961,FALSE,,,,,,,,,,,,"
  ],
  "category": "clinical-note",
  "userInput": "How is the patient doing?",
  "previousSummary": ""
}

def stream_response(url):
    with requests.post(url, json=payload, stream=True) as response:
        print(response.headers)
        if response.status_code == 200:
            
            # Iterate over the response content     
            buffer = b''
            for chunk in response.iter_content(decode_unicode=True):
                buffer += chunk

                try:
                    # Once the whole json chunk comes, we can decode it and get the body
                    parsed_json = json.loads(buffer.decode('utf-8'))
                    print(parsed_json.get("body"), end='')
                    buffer = b''
                except: 
                    continue
        else: 
            print(f"Failed to connect, status code: {response.status_code}")

stream_response(apigateway_url)

{'Date': 'Wed, 21 Aug 2024 14:39:46 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '45863', 'Connection': 'keep-alive', 'x-amzn-RequestId': '375c19cf-4a93-42d4-831c-57cf2bfa656a', 'x-amzn-Remapped-x-amzn-RequestId': 'db48f4ba-0635-44de-8a30-361fbe9ff60a', 'x-amzn-Remapped-Connection': 'keep-alive', 'x-amz-apigw-id': 'c3RV4HkUIAMEYBA=', 'X-Amzn-Trace-Id': 'root=1-66c5fc25-0f73c0503e81425727758e84;parent=742a2b3ac54a8065;sampled=0;lineage=44f17ad1:0', 'x-amzn-Remapped-Date': 'Wed, 21 Aug 2024 14:39:35 GMT'}
Based on the clinical notes provided, I can summarize the patient's condition as follows:
1. <b>Overall Status</b>:     <ul>         <li>The patient is generally stable across multiple dialysis sessions</li>         <li>No major complaints or issues reported</li>     </ul>
2. <b>Dialysis Sessions</b>:     <ul>         <li>Patient completed a 4-hour hemodialysis session without complications</li>         <li>Pre-dialysis notes indicate patient was stable before tre

In [19]:
'''
Requests the local fastAPI backend that will be deployed using LWA
'''

import requests

payload = {
  "responseData": [
    "id,note,type,details,fhir_id,id_patient,created_at,modified_at,created_by,modified_by,id_tenant,id_branch,id_dialysis,manually_created,hd_reading_id,lab_order_id,,,,,,,,,,",
    "11989,'pt stable, complete 4 hour hd, no complaint',HD,Post-dialysis,694417,1951,54:09.4,,233,,100,10000,9042,FALSE,,,,,,,,,,,,",
    "11976,'pt stable, no sign of infection, no sign of covid',HD,Pre-dialysis,694027,1951,02:56.5,,233,,100,10000,9042,FALSE,,,,,,,,,,,,",
    "11859,'pt stable and afebrile\nno sign and symptom of infection seen\nno sign and symptom of edema seen\npt on profiling linear, start at 1.15L/hr',HD,Pre-dialysis,690742,1951,19:14.8,,172,,100,10000,8961,FALSE,,,,,,,,,,,,"
  ],
  "category": "clinical-note",
  "userInput": "How is the patient doing?",
  "previousSummary": ""
}

def stream_response(url):
    with requests.post(url, json=payload, stream=True) as response:
        if response.status_code == 200:
            # Iterate over the response content as it arrives
            for chunk in response.iter_content(decode_unicode=True):
                if chunk:
                    print(chunk, end='', flush=True)
        else: 
            print(f"Failed to connect, status code: {response.status_code}")

stream_response('http://127.0.0.1:8080')

Failed to connect, status code: 422
